In [1]:
## testing mikolaj continue

# Setup

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch


ModuleNotFoundError: No module named 'openai'

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


# Load file

In [ ]:
file = 'data/pdfs/ISLRv2_website.pdf'

loader = PyPDFLoader(file)
document = loader.load()
document[99]

Document(page_content='90 3. Linear Regression\n0 50 100 150200 600 1000 1400IncomeBalance0 50 100 150200 600 1000 1400IncomeBalancestudentnon−student\nFIGURE 3.7.For theCreditdata, the least squares lines are shown for pre-diction ofbalancefromincomefor students and non-students.Left:The model(3.34) was ﬁt. There is no interaction betweenincomeandstudent.Right:Themodel (3.35) was ﬁt. There is an interaction term betweenincomeandstudent.takes the formbalancei≈β0+β1×incomei+/braceleft⎪iggβ2ifith person is a student0 ifith person is not a student=β1×incomei+/braceleft⎪iggβ0+β2ifith person is a studentβ0ifith person is not a student.(3.34)Notice that this amounts to ﬁtting two parallel lines to the data, one forstudents and one for non-students. The lines for students and non-studentshave diﬀerent intercepts,β0+β2versusβ0, but the same slope,β1. Thisis illustrated in the left-hand panel of Figure 3.7. The fact that the linesare parallel means that the average eﬀect onbalanceof a one-unit 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_documents(document)
splits[99]

Document(page_content='FIGURE 2.9.Left:Data simulated fromf, shown in black. Three estimates offare shown: the linear regression line (orange curve), and two smoothing splineﬁts (blue and green curves).Right:Training MSE (grey curve), test MSE (redcurve), and minimum possible test MSE over all methods (dashed line). Squaresrepresent the training and test MSEs for the three ﬁts shown in the left-handpanel.since the training MSE and the test MSE appear to be closely related.Unfortunately, there is a fundamental problem with this strategy: thereis no guarantee that the method with the lowest training MSE will alsohave the lowest test MSE. Roughly speaking, the problem is that manystatistical methods speciﬁcally estimate coeﬃcients so as to minimize thetraining set MSE. For these methods, the training set MSE can be quitesmall, but the test MSE is often much larger.Figure 2.9 illustrates this phenomenon on a simple example. In the left-hand panel of Figure 2.9, we have generated observatio

# Convert the pages into embeddings and store in vector store

In [ ]:
embeddings = OpenAIEmbeddings()


In [ ]:
db = DocArrayInMemorySearch.from_documents(
    splits, 
    embeddings
)

2023-08-03 14:42:18.233247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 14:42:20.724440: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 14:42:25.820079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-03 14:42:25.822743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# Query a store

## Pose a query, find pages similar to query

In [ ]:
query = """Explain bias-variance tradeoff for a linear regression model."""

In [ ]:
similar_pages = db.similarity_search(query, limit=10)
similar_pages

[Document(page_content='bias-variancetrade-oﬀ in mind. In this book we explore methods that are extremelyﬂexible and hence can essentially eliminate bias. However, this does notguarantee that they will outperform a much simpler method such as linearregression. To take an extreme example, suppose that the truefis linear.In this situation linear regression will have no bias, making it very hardfor a more ﬂexible method to compete. In contrast, if the truefis highlynon-linear and we have an ample number of training observations, thenwe may do better using a highly ﬂexible approach, as in Figure 2.11. InChapter 5 we discuss cross-validation, which is a way to estimate the testMSE using the training data.', metadata={'source': 'data/pdfs/ISLRv2_website.pdf', 'page': 46}),
 Document(page_content='2.2 Assessing Model Accuracy 35one of these data points may cause the estimateˆfto change considerably.In contrast, the orange least squares line is relatively inﬂexible and has lowvariance, because

## Retriever

In [ ]:
retriever = db.as_retriever()


## Query the stored pages

In [ ]:
llm = ChatOpenAI(temperature = 0.0)

qpages = "".join([similar_pages[i].page_content for i in range(len(similar_pages))])


In [ ]:
response = llm.call_as_llm(f"{qpages} Question: {query}") 


In [ ]:
display(Markdown(response))

The bias-variance tradeoff for a linear regression model refers to the relationship between the model's bias and variance. 

Bias refers to the error that is introduced by approximating a real-life problem, which may be extremely complicated, by a much simpler model. In the case of linear regression, it assumes a linear relationship between the dependent variable and the independent variables. If the true relationship is not linear, using linear regression will result in bias.

Variance, on the other hand, refers to the amount by which the estimated regression line would change if we estimated it using a different training dataset. If a model has high variance, small changes in the training data can result in large changes in the estimated regression line.

The tradeoff occurs because as we increase the flexibility of the model, such as by using more complex regression models, the bias tends to decrease but the variance tends to increase. This means that more flexible models can better capture the true relationship between the variables, reducing bias, but they are also more sensitive to the specific training data, increasing variance.

In the case of linear regression, it is a relatively simple and less flexible model. It assumes a linear relationship between the variables and has low variance. However, if the true relationship is highly non-linear, linear regression will have high bias and may not accurately capture the true relationship.

Therefore, the bias-variance tradeoff for a linear regression model is that it has low variance but potentially high bias, especially if the true relationship is non-linear.